In [ ]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
import seaborn as sns
import os

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
dff = pd.read_excel('/Users/farhanhabibie/Desktop/Farhan Thesis Code /UG-Project-Farhan/Indonesia Macro Dataset.xlsx')
dff

In [ ]:
"""
Function that loads and prepares the dataframe. 

Converts the data into time-series readable.
"""
def initialFormat (filepath, indicatorcode):
    #Reads the file and creates a dataframe from it
    df = pd.read_excel(filepath)
    
    #Choose what to forecast using indicator code
    df_icode = df.loc[df['Indicator Code'] == indicatorcode]
    
    #Dropping these columns as they are not needed for the forecast
    df_icode = df_icode.drop(columns=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'])
    
    #Swap axis so it is in the proper format
    df_formatted = df_icode.swapaxes("index", "columns")
    
    #Renaming column name to 'values' to make reference easier
    for col_names in df_formatted.columns:
        name = df_formatted.rename(columns={col_names : "thing"})
        return name
    
    return df_formatted

#9 Lines Written by me

In [ ]:
"""
Function to see how many NA values in the dataset, written by me

"""

def null_percent(data):
    
    percentage = (data.isnull().sum()/len(data))*100
    
    print('Percentage of NA/NAN in this set is: ', percentage)
    
    #3 Lines Written by me

In [ ]:
"""
Refactored completely from:
https://towardsdatascience.com/time-series-from-scratch-train-test-splits-and-evaluation-metrics-4fd654de1b37

Future versions of this function uses this same logic however modifies the function slightly.
"""
def train_test_split(data, test_size):
    
    train = data[:-test_size]
    test = data[-test_size:]
    
    return train, test

#4 Lines copied from guide

In [ ]:
"""
Visualise the split
Also from, but is very generic code:
https://towardsdatascience.com/time-series-from-scratch-train-test-splits-and-evaluation-metrics-4fd654de1b37
"""
def plot_train_test(train_set, test_set):
    ax = plt.plot(train_set, label='Train set')
    ax = plt.plot(test_set, label='Test set')
    ax = plt.title('GDP Split')
    
    return ax

#4 Lines copied from guide

In [ ]:
"""
This function was refactored from:

https://towardsdatascience.com/predicting-electricity-consumption-with-xgbregressor-a11b71104754

https://towardsdatascience.com/xgboost-for-time-series-forecasting-dont-use-it-blindly-9ac24dc5dfa9

These two guides are written by the same user (Michael Grogan)
"""

def create_dataset(df, previous=1):
    dataX, dataY = [], []
    for i in range (len(df)-previous-1):
        a = df[i:(i+previous), 0]
        dataX.append(a)
        dataY.append(df[i+previous, 0])
    return np.array(dataX), np.array(dataY)

#7 Lines Taken directly from guide

In [ ]:
"""
line 11 (MDA) reused from: https://gist.github.com/bshishov/5dc237f59f019b26145648e2124ca1c9

line 8 (MAPE) adapted from: https://www.statology.org/mape-python/

in future version line 11 (MDA), the code structure has been modified slightly.
"""
def performance_metrics(y_test, y_pred):
    rmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
    mape = np.mean(np.abs((y_test - y_pred) / y_test )) *100
    mae = metrics.mean_absolute_error(y_test, y_pred)
    #r2 = metrics.r2_score(y_test, y_pred)
    mda = np.mean((np.sign(y_test[1:] - y_test[:-1]) == np.sign(y_pred[1:] - y_pred[:-1])).astype(int))
    mean = np.mean(y_test)
    si = (rmse/mean)*100
    
    print("RMSE: ", rmse)
    print("MAPE: ", mape)
    print("MAE: ", mae)
    print("MDA: ", mda)
    print("Mean of actual: ", mean)
    print("Scatter Index: ", si)
    
#13 Lines. 3 From documentation, 1 from github, 9 written by me

In [ ]:
"""
Written by me but utilised function calls to statsmodels library
"""

def plot_ACF(data, lag):
    acfData = data.dropna()
    print (len(acfData))
    plot_acf(acfData, lags=lag)

In [ ]:
data = initialFormat('/Users/farhanhabibie/Desktop/Farhan Thesis Code /UG-Project-Farhan/Indonesia Macro Dataset.xlsx', 
                     'NY.GDP.MKTP.KD.ZG')

null_percent(data)
data.plot(color='green')
#1 Line Written by me

In [ ]:
plot_ACF(data, 28)

#1 Line Written by me

In [ ]:
#Here we split, with the train set being 80% and test set being 20%
train, test = train_test_split(data, len(data) - int(len(data) * 0.8))
train, test = train.dropna(), test.dropna()

In [ ]:
plot_train_test(train, test)

In [ ]:
"""
This part was refactored from:

https://towardsdatascience.com/predicting-electricity-consumption-with-xgbregressor-a11b71104754

https://towardsdatascience.com/xgboost-for-time-series-forecasting-dont-use-it-blindly-9ac24dc5dfa9

These two guides are written by the same user (Michael Grogan)
"""
lookback = 1

X_train, Y_train = create_dataset(train.to_numpy(), lookback)
X_test, Y_test = create_dataset(test.to_numpy(), lookback)

In [ ]:
"""
Written by me but uses function calls to xgb library
"""

model = XGBRegressor(objective='reg:squarederror', n_estimators=1000, max_depth=6, learning_rate=0.3)
model.fit(X_train, Y_train)
y_pred = model.predict(X_test)

In [ ]:
"""
Code written by me, however this method (line 9) was also used in a previous Introduction to AI Course I partook in,
at City University of London

Link to repository provided: 
https://github.com/LabiKSV/intro-to-ai-farhan-labi/blob/main/Linear%20Regression%20Label%20Encoder.ipynb
"""

df_compare = pd.DataFrame({'Actual' : Y_test, 'Predicted' : y_pred})
df_compare.plot(title='GDP Actual vs Predicted')
performance_metrics(Y_test, y_pred)

In [ ]:
"""
Starting here is the hyperparameter optimisation for univariate forecasting

This piece of code was reused from https://towardsdatascience.com/xgboost-fine-tune-and-optimize-your-model-23d996fab663

Fitted for my dataset
"""

def tune_n_estimator(X, y):
    hyperparam = {'max_depth': [3,6,10],
                 'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2, 0.3],
                 'n_estimators': [5,10,50,100,1000]}
    
    model = XGBRegressor()
    
    gscv = GridSearchCV(estimator=model, param_grid=hyperparam,
                       scoring='neg_mean_squared_error',
                       verbose=1)
    
    gscv.fit(X, y)
    
    print("Best hyperparam: ", gscv.best_params_)
    
#6 Lines from guide, modified by me

In [ ]:
"""
Commented out because not needed. Uncomment at need.
"""

#tune_n_estimator(X_train, Y_train)

In [ ]:
"""71 Lines of code, 10 import statements"""